In this demo, we'll show how to use `pyia` to transform the raw Gaia data in various ways. For example, to extract Astropy coordinate objects, to extinction-correct the photometry, and to reconstruct and sample from the Gaia error distribution (i.e. the full astrometric covariance matrix). 

We'll use [astropy](http://astropy.org), [pyia](http://pyia.readthedocs.io), and [gala](http://gala.adrian.pw) for this demo, which can be installed with (but make sure you have installed numpy, scipy, astropy, and cython):

* `pip install pyia`
* `pip install git+https://github.com/adrn/gala` 

(we install the developer version of `gala`)

To make sure you have everything installed (and have anaconda installed), execute this in your terminal:
```
conda install numpy scipy astropy cython
pip install astroquery pyia pyyaml
pip install git+https://github.com/adrn/gala
```

In [ ]:
# Third-party
import astropy.coordinates as coord
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

import gala.coordinates as gc
import gala.dynamics as gd
import gala.potential as gp
from gala.units import galactic
from pyia import GaiaData

As an example, we'll download a set of stars around the Hyades cluster. We can use `astropy.coordinates` to retrieve a coordinate object given the name (which queries Simbad for us):

In [ ]:
c = coord.SkyCoord.from_name('Hyades')

To get our hands on some Gaia data, we'll use the `.from_query()` classmethod to execute an ADQL query in the Gaia science archive, and retrieve the results as an astropy Table object. The query below will take a few seconds to execute, and will output a bunch of warnings - it is safe to ignore the warnings!

In [ ]:
g = GaiaData.from_query("""SELECT * FROM gaiadr2.gaia_source 
                           WHERE parallax > 5. AND 
                           1=CONTAINS(
                               POINT('ICRS', ra, dec),
                               CIRCLE('ICRS', {c.ra.degree}, {c.dec.degree}, 5.))"""
                        .format(c=c))

How many objects were returned by this query?

In [ ]:
len(g)

The data are internally stored as an astropy table via the `.data` attribute:

In [ ]:
g.data[:4]

But the `GaiaData` object provides a number of convenience methods and attributes that will add units to Gaia quantities, and help in working with the Gaia data. For example, this includes simple things like compute the distance by inverting the parallax:

In [ ]:
g.distance

Or computing the distance modulus:

In [ ]:
g.distmod

But also includes more advanced things, as we'll see later. Let's start by looking at the distance distribution of stars in our query response:

In [ ]:
plt.hist(g.get_distance().to_value(u.pc), bins='auto')
plt.xlabel(r'distance, $1/\varpi$ [pc]');

The Hyades cluster pops out clearly as a peak around distance ~ 45–50 pc. Let's now look at the 2D proper motion distribution for stars within 100 pc and see if the Hyades shows up:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.hist2d(g.pmra.value[g.distance < 100*u.pc], 
          g.pmdec.value[g.distance < 100*u.pc],
          bins=np.arange(-150, 150+1e-3, 5),
          cmap='Greys');
ax.set_xlabel('pmra [{:latex_inline}]'.format(g.pmra.unit))
ax.set_ylabel('pmdec [{:latex_inline}]'.format(g.pmdec.unit));

The Hyades again show up as a large over-density around (110, -25) mas/yr! Let's now combine some selections on distance and proper motion to do a rough selection of Hyades cluster members that have measured radial velocities in Gaia:

In [ ]:
pm_mask = ((g.pmra > 50*u.mas/u.yr) & 
           (g.pmra < 150*u.mas/u.yr) & 
           (g.pmdec < 0*u.mas/u.yr) &
           (g.pmdec > -50*u.mas/u.yr))
dist_mask = (g.distance < 100*u.pc) & (g.parallax_over_error > 8)
has_rv = np.isfinite(g.radial_velocity)

In [ ]:
g_hyades = g[pm_mask & dist_mask & has_rv]
len(g_hyades)

So there are about 105 stars that match out (very rough, restrictive) cuts. Let's look at the color-magnitude diagram for these stars:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.plot(g_hyades.phot_bp_mean_mag - g_hyades.phot_rp_mean_mag,
        g_hyades.phot_g_mean_mag - g_hyades.distmod, 
        marker='o', ms=3, ls='none', color='k')
ax.set_xlim(0, 3)
ax.set_ylim(10, -1)
ax.set_xlabel('$G_{BP}-G_{RP}$')
ax.set_ylabel('$M_G$');

The beautifully line up on a thin main sequence in the CMD, so it looks like our selection is pretty pure.

We now might want to look at the distribution of Hyades stars in Galactic cartesian coordinates. We can handle these coordinate frame and representation transformations using the [astropy.coordinates](http://docs.astropy.org/en/latest/coordinates/index.html) transformation machinery. To do this, we first have to retrieve an astropy `SkyCoord` object for our data. We can construct this object automatically with the `GaiaData` object using the `.get_skycoord()` method:

In [ ]:
hyades_c = g_hyades.get_skycoord()

And to transform to Galactocentric, Cartesian coordinates:

In [ ]:
galcen = hyades_c.transform_to(coord.Galactocentric(galcen_distance=8.1*u.kpc))

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(8, 8),
                         sharex='col', sharey='row')

style = dict(marker='o', ms=2., ls='none', color='k')
axes[0, 0].plot(galcen.v_x, galcen.v_y, **style)
axes[1, 0].plot(galcen.v_x, galcen.v_z, **style)
axes[1, 1].plot(galcen.v_y, galcen.v_z, **style)
axes[0, 1].set_visible(False)

axes[0, 0].set_xlim(-40, -25)
axes[0, 0].set_ylim(205, 220)
axes[1, 0].set_ylim(0, 15)
axes[1, 1].set_xlim(205, 220)

for ax in axes.flat:
    ax.set_aspect('equal')
    
axes[0, 0].set_ylabel('$v_y$')
axes[1, 0].set_ylabel('$v_z$')
axes[1, 0].set_xlabel('$v_x$')
axes[1, 1].set_xlabel('$v_y$')
    
fig.tight_layout()

However, these are just the point estimates reported in the Gaia catalog. We may want to propagate the (correlated) astrometric and radial velocity uncertainties for each source through this transformation. To do that, we can first generate samples from the error distribution and then transform the samples. To generate samples from the error distribution, we can use the `.get_error_samples()` method, here to generate 1000 random samples from each source's error distribution:

In [ ]:
g_samples = g_hyades.get_error_samples(size=1000)

In [ ]:
g_hyades.pmra.shape

In [ ]:
g_samples.pmra.shape

So, for each of the 105 Hyades-selected stars, we now have 1000 error samples as well. We transform these to Galactocentric coordinates in the same way:

In [ ]:
hyades_samples_c = g_samples.get_skycoord()
galcen_samples = hyades_samples_c.transform_to(coord.Galactocentric(galcen_distance=8.1*u.kpc))

We can now look at the typical velocity uncertainties for each source in each of the velocity components:

In [ ]:
bins = np.linspace(0, 1.5, 21)
plt.hist(np.std(galcen_samples.v_x, axis=1), 
         bins=bins, label=r'$v_x$', alpha=0.4)
plt.hist(np.std(galcen_samples.v_y, axis=1), 
         bins=bins, label=r'$v_y$', alpha=0.4)
plt.hist(np.std(galcen_samples.v_z, axis=1), 
         bins=bins, label=r'$v_z$', alpha=0.4)
plt.xlabel(r'$\sigma_{v}$ ' + '[{:latex_inline}]'.format(u.km/u.s))

However, the above only shows the uncorrelated uncertainties. The error samples will generically be correlated, e.g.:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.plot(galcen_samples.v_x[0], 
        galcen_samples.v_z[0],
        marker='o', ms=2., color='k', alpha=0.4, ls='none')
ax.set_xlabel(r'$v_x$')
ax.set_ylabel(r'$v_z$')

With Galactocentric coordinates for these sources, and an assumed model for the Milky Way, we can also numerically integrate the orbits of these stars and look at their Galactic orbits. We'll use the default Milky Way model implemented in Gala, and integrate the orbits of the Hyades stars backwards in time for 400 Myr:

In [ ]:
potential = gp.MilkyWayPotential()

In [ ]:
# transform to initial conditions
w0 = gd.PhaseSpacePosition(galcen.data)
orbits = potential.integrate_orbit(w0, dt=-0.1*u.Myr, n_steps=4000)

In [ ]:
orbits.shape

Plot the orbits:

In [ ]:
_ = orbits.plot(marker='', linestyle='-', 
                linewidth=0.5, alpha=0.5, color='k')

With the orbit objects, we can also compute the eccentricities of the orbits in the Galaxy:

In [ ]:
ecc = orbits.eccentricity()

In [ ]:
ecc.shape

In [ ]:
plt.hist(ecc, bins='auto');
plt.xlabel('eccentricity')